In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
mnist=tf.keras.datasets.mnist
(train_images,train_labels),(test_images,test_labels)=mnist.load_data()

train_labels=tf.one_hot(train_labels,depth=10).numpy()
test_labels=tf.one_hot(test_labels,depth=10).numpy()

train_images=train_images/255.0
test_images=test_images/255.0

2023-03-11 01:16:18.255192: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(units=64,kernel_initializer='normal',activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(units=32,kernel_initializer='normal',activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10,activation=tf.nn.softmax))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 64)                50240     
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 10)                330       
                                                                 
Total params: 52,650
Trainable params: 52,650
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [6]:
train_epochs=5
batch_size=50
trained_model=model.fit(train_images,train_labels,validation_split=0.2,epochs=train_epochs,batch_size=batch_size,verbose=2)

Epoch 1/5
960/960 - 2s - loss: 0.3844 - accuracy: 0.8928 - val_loss: 0.1906 - val_accuracy: 0.9445 - 2s/epoch - 2ms/step
Epoch 2/5
960/960 - 1s - loss: 0.1703 - accuracy: 0.9502 - val_loss: 0.1502 - val_accuracy: 0.9560 - 1s/epoch - 1ms/step
Epoch 3/5
960/960 - 1s - loss: 0.1242 - accuracy: 0.9630 - val_loss: 0.1354 - val_accuracy: 0.9607 - 1s/epoch - 1ms/step
Epoch 4/5
960/960 - 1s - loss: 0.0990 - accuracy: 0.9705 - val_loss: 0.1150 - val_accuracy: 0.9661 - 1s/epoch - 1ms/step
Epoch 5/5
960/960 - 1s - loss: 0.0797 - accuracy: 0.9753 - val_loss: 0.1094 - val_accuracy: 0.9672 - 1s/epoch - 1ms/step


In [7]:
trained_model.history

{'loss': [0.3844381868839264,
  0.17030492424964905,
  0.12423007935285568,
  0.09903085976839066,
  0.07970244437456131],
 'accuracy': [0.8928124904632568,
  0.950208306312561,
  0.9629583358764648,
  0.9704583287239075,
  0.9752916693687439],
 'val_loss': [0.1906021535396576,
  0.15023893117904663,
  0.13536381721496582,
  0.11500296741724014,
  0.10939214378595352],
 'val_accuracy': [0.9445000290870667,
  0.9559999704360962,
  0.9607499837875366,
  0.9660833477973938,
  0.9671666622161865]}